In [3]:
import os
from langchain_community.graphs import Neo4jGraph
from dotenv import load_dotenv

load_dotenv(".env")

graph = Neo4jGraph(
    url=os.environ["NEO4J_URI"],
    username=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"],
)

In [4]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()

q_load_articles = """
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/dcarpintero/generative-ai-101/main/dataset/synthetic_articles.csv' 
AS row 
FIELDTERMINATOR ';'
MERGE (a:Article {title:row.Title})
SET a.abstract = row.Abstract,
    a.publication_date = date(row.Publication_Date)
FOREACH (researcher in split(row.Authors, ',') | 
    MERGE (p:Researcher {name:trim(researcher)})
    MERGE (p)-[:PUBLISHED]->(a))
FOREACH (topic in [row.Topic] | 
    MERGE (t:Topic {name:trim(topic)})
    MERGE (a)-[:IN_TOPIC]->(t))
"""

graph.query(q_load_articles)

[]

In [5]:
graph.refresh_schema()
print(graph.get_schema)

Node properties:
Article {title: STRING, abstract: STRING, publication_date: DATE}
Researcher {name: STRING}
Topic {name: STRING}
Relationship properties:

The relationships:
(:Article)-[:IN_TOPIC]->(:Topic)
(:Researcher)-[:PUBLISHED]->(:Article)


In [ ]:
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    url=os.environ["NEO4J_URI"],
    username=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"],
    index_name="articles",
    node_label="Article",
    text_node_properties=["topic", "title", "abstract"],
    embedding_node_property="embedding",
)

In [7]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

vector_qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(), chain_type="stuff", retriever=vector_index.as_retriever())

In [8]:
r = vector_qa.invoke(
    {
        "query": "which articles discuss how AI might affect our daily life? include the article titles and abstracts."
    }
)
print(r["result"])

The articles that discuss how AI might affect our daily life are:

1. Title: The Societal Implications of Advanced AI: A Multidisciplinary Analysis
Abstract: Our study brings together experts from various fields to analyze the potential long-term impacts of advanced AI on society, economy, and culture.

2. Title: Ethical Considerations in AI Development
Abstract: We explore the ethical implications of rapid AI advancement and propose guidelines for responsible development.


In [13]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI

graph.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm=ChatOpenAI(temperature=0, model_name="gpt-4o"),
    qa_llm=ChatOpenAI(temperature=0, model_name="gpt-4o"),
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True
)

In [14]:
# the answer should be '7'
cypher_chain.invoke({"query": "How many articles has published Emily Chen?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (r:Researcher {name: "Emily Chen"})-[:PUBLISHED]->(a:Article)
RETURN COUNT(a) AS numberOfArticles

Full Context:
[{'numberOfArticles': 7}]

> Finished chain.


{'query': 'How many articles has published Emily Chen?',
 'result': 'Emily Chen has published 7 articles.'}

In [15]:
cypher_chain.invoke(
    {"query": "are there any pair of researchers who have published more than three articles together?"}
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (r1:Researcher)-[:PUBLISHED]->(a:Article)<-[:PUBLISHED]-(r2:Researcher)
WHERE r1 <> r2
WITH r1, r2, COUNT(a) AS sharedArticles
WHERE sharedArticles > 3
RETURN r1.name, r2.name, sharedArticles

Full Context:
[{'r1.name': 'Robert Taylor', 'r2.name': 'Emily Chen', 'sharedArticles': 4}, {'r1.name': 'David Johnson', 'r2.name': 'Emily Chen', 'sharedArticles': 4}, {'r1.name': 'Emily Chen', 'r2.name': 'David Johnson', 'sharedArticles': 4}, {'r1.name': 'Emily Chen', 'r2.name': 'Robert Taylor', 'sharedArticles': 4}]

> Finished chain.


{'query': 'are there any pair of researchers who have published more than three articles together?',
 'result': 'Yes, Robert Taylor and Emily Chen, as well as David Johnson and Emily Chen, have published more than three articles together.'}